## Preparing Data


In [2]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

We then load the IMDb dataset.

In [3]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

print(vars(train_data.examples[0]))

#print(vars(train_data.examples[i]).get('text').reverse())
for i in train_data:
  i.text=i.text[::-1]

print(vars(train_data.examples[0]))

aclImdb_v1.tar.gz:   0%|          | 98.3k/84.1M [00:00<01:26, 970kB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 51.6MB/s]


{'text': ['After', 'losing', 'the', 'Emmy', 'for', 'her', 'performance', 'as', 'Mama', 'Rose', 'in', 'the', 'television', 'version', 'of', 'GYPSY', ',', 'Bette', 'won', 'an', 'Emmy', 'the', 'following', 'year', 'for', 'BETTE', 'MIDLER', ':', 'DIVA', 'LAS', 'VEGAS', ',', 'a', 'live', 'concert', 'special', 'filmed', 'for', 'HBO', 'from', 'Las', 'Vegas', '.', 'Midler', ',', 'who', 'has', 'been', 'performing', 'live', 'on', 'stage', 'since', 'the', '1970', "'s", ',', 'proves', 'that', 'she', 'is', 'still', 'one', 'of', 'the', 'most', 'electrifying', 'live', 'performers', 'in', 'the', 'business', '.', 'From', 'her', 'opening', 'number', ',', 'her', 'classic', '"', 'Friends', '"', ',', 'where', 'she', 'descends', 'from', 'the', 'wings', 'atop', 'a', 'beautiful', 'prop', 'cloud', ',', 'Bette', 'commands', 'the', 'stage', 'with', 'style', 'and', 'charisma', 'from', 'a', 'rap', '-', 'styled', 'number', 'called', '"', 'I', 'Look', 'Good', '"', 'she', 'then', 'proves', 'that', 'she', 'has', 'a', 

In [4]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:26, 2.23MB/s]                           
100%|█████████▉| 399458/400000 [00:18<00:00, 22320.10it/s]

In [19]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

## Build the Model



In [20]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.fc2 = nn.Linear(512, 100)
        self.fc3 = nn.Linear(100,512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        for i in range(3):
          packed_output, (hidden,cell) = self.rnn(packed_embedded)
          packed_embedded = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
          packed_embedded = packed_embedded.unsqueeze(0)
          packed_embedded = self.fc2(packed_embedded)
          packed_embedded = self.relu(packed_embedded)
          #print(f"Packed_EMBEDDED : {packed_embedded.shape}")


        hidden = self.dropout(self.relu(self.fc3(packed_embedded)))
        #hidden = self.fc3(packed_embedded)
        #packed_output, (hidden, cell) = self.rnn(packed_embedded) #Co
        
        #unpack sequence
        #output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output) #Co

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        #hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)) #Co
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [28]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [29]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,913,869 trainable parameters


In [30]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [31]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2307, -0.7766,  0.9134,  ..., -0.7217,  0.0664, -0.7596],
        [-0.5702, -0.0414,  1.0065,  ..., -0.5042, -1.1177,  0.1139],
        [-1.5217, -1.2295,  2.3400,  ..., -0.1066, -1.5500,  1.2515]])

In [32]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2307, -0.7766,  0.9134,  ..., -0.7217,  0.0664, -0.7596],
        [-0.5702, -0.0414,  1.0065,  ..., -0.5042, -1.1177,  0.1139],
        [-1.5217, -1.2295,  2.3400,  ..., -0.1066, -1.5500,  1.2515]])


## Train the Model

In [33]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [34]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [35]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [36]:

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths.cpu()).squeeze()
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths.cpu()).squeeze()
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [38]:

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [39]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 38s
	Train Loss: 0.629 | Train Acc: 63.09%
	 Val. Loss: 0.568 |  Val. Acc: 70.47%
Epoch: 02 | Epoch Time: 0m 39s
	Train Loss: 0.539 | Train Acc: 73.01%
	 Val. Loss: 0.623 |  Val. Acc: 64.93%
Epoch: 03 | Epoch Time: 0m 39s
	Train Loss: 0.409 | Train Acc: 81.21%
	 Val. Loss: 0.352 |  Val. Acc: 84.45%
Epoch: 04 | Epoch Time: 0m 40s
	Train Loss: 0.280 | Train Acc: 88.46%
	 Val. Loss: 0.312 |  Val. Acc: 88.04%
Epoch: 05 | Epoch Time: 0m 41s
	Train Loss: 0.221 | Train Acc: 91.54%
	 Val. Loss: 0.277 |  Val. Acc: 88.78%


In [40]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.340 | Test Acc: 85.51%


In [41]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [44]:
predict_sentiment(model, "This film is terrible")

0.002912783296778798

In [45]:
predict_sentiment(model, "This film is great")

0.874282717704773